# Merges

Merge lets us combine multiple data frames. The general syntax is as follows to merge pandas dataframes df_1 and df_2:
    
    df_1.merge(df_2, how = ["inner", "outer", "left"] , left_on = left_column_name, right_on = right_column_name)

the example above shows the three main inputs to the how arguments.  When actually using a merge you should only specify one of these three values for the how arguments. The how argument specifies the type of merge and the left_on and right_on arguments tells pandas which columns to match up from each of the two dataframes when deciding which rows to keep.  In the example above, df_1 is the left dataframe and df_2 is the right dataframe.

In [1]:
import pandas as pd

In [17]:
df_doc = pd.DataFrame( {"Doc_ID": [1,2,3,4], "Name": ["A","B","C","D"], "Degree": ["MD","MBBS","MD","MBBS"]})
df_doc

,Doc_ID,Name,Degree
0,1,A,MD
1,2,B,MBBS
2,3,C,MD
3,4,D,MBBS


In [18]:
df_doc_visits = pd.DataFrame({"Doc_ID": [1,2,5,6,7], "Name": ["E","F","G","H","I"], "Year": [2010,2011,2012,2013,2014]})
df_doc_visits

,Doc_ID,Name,Year
0,1,E,2010
1,2,F,2011
2,5,G,2012
3,6,H,2013
4,7,I,2014


For every visit, I would like to create a table with the doctor that saw the patient, if one exists. We will merge on the doctor_id column with an inner merge.  With an inner merge, if a match isn't found, then we do not include the row. If the two columns that you want to merge on have the same name, then you can simply specify the on argument instead of separate left_on and right_on arguments taking the same value.

In [19]:
df_merge = df_doc_visits.merge(df_doc, how="inner", on="Doc_ID")

df_merge.rename(columns= {"Name_x": "Patient_Name", "Name_y": "Doctor_Name" }, inplace = True)

df_merge.loc[:, ["Doctor_Name", "Year", "Patient_Name"]]

,Doctor_Name,Year,Patient_Name
0,A,2010,E
1,B,2011,F


Now lets see how we can use a left merge to keep these visits that don't have a matching doctor.

In [22]:
df_left_merge = df_doc_visits.merge(df_doc, how="left", on="Doc_ID")
df_left_merge

,Doc_ID,Name_x,Year,Name_y,Degree
0,1,E,2010,A,MD
1,2,F,2011,B,MBBS
2,5,G,2012,NaN,NaN
3,6,H,2013,NaN,NaN
4,7,I,2014,NaN,NaN


Notice that in this example, we get a row for every visit, but for visits that don't have a matching doctor_id we get NaNs from the doctors table.

Finally, let's see an example of an outer merge.  Consider the following two dataframes.

In [23]:
df1 = pd.DataFrame({"week": [1,2,3,4], "rushpoints": [10,20,30,40]})
df1

,week,rushpoints
0,1,10
1,2,20
2,3,30
3,4,40


In [24]:
df2 = pd.DataFrame({"week": [2,3,5], "rushpoints": [100,200,300]})
df2

,week,rushpoints
0,2,100
1,3,200
2,5,300


In [29]:
df_final= df1.merge(df2, how="outer", on="week")
df_final

,week,rushpoints_x,rushpoints_y
0,1,10.0,NaN
1,2,20.0,100.0
2,3,30.0,200.0
3,4,40.0,NaN
4,5,NaN,300.0


Lets say I wanted to combine the records and put a zero if one of the points categories did not exists for the given week. In this case I want to merge on Week, but each table has weeks that other doesn't have so a left merge will delete rows no matter which dataframe I choose as the left one. 

In [28]:
df_final= df1.merge(df2, how="outer", on="week")

df_final.fillna(0, inplace=True)

df_final

,week,rushpoints_x,rushpoints_y
0,1,10.0,0.0
1,2,20.0,100.0
2,3,30.0,200.0
3,4,40.0,0.0
4,5,0.0,300.0


Notice that the resulting data frame has a row for each week!